In [143]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [144]:
Hot_df = pd.read_excel('Datos/Ejercicio4.xlsx', 'Hot',skiprows = 1,index_col=0)
Hot_df

,0,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,,
H1,0,0,610,10,270,140,0,0,0,0,0
H2,0,0,0,0,270,140,350,110,420,0,0
H3,0,0,0,0,0,0,0,880,0,0,0
H4,0,0,0,0,0,0,0,0,1260,600,0


E igualmente de las frías.

In [145]:
Cold_df = pd.read_excel('Datos/Ejercicio4.xlsx', 'Cold',skiprows = 1,index_col=0)
Cold_df

,0,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,,
C1,0,0.000,0.000,25.000,675.000,350,875,275,1050,500,250
C2,0,1828.597,1828.597,29.977,809.379,0,0,0,0,0,0


In [146]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
s=hotst.copy()
s.insert(0,'S')
w=coldst.copy()
w.insert(0,'W')

model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.s=Set(initialize=s)
model.S=Set(initialize='S')
model.w=Set(initialize=w)
model.W=Set(initialize='W')
model.k=Set(initialize=Hot_df.columns)

model.hot_st=model.i | model.S
model.cold_w=model.j | model.W


Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [147]:
intmax=max(Hot_df.columns)

Creamos ahora las variables del modelo. Hay i potenciales aportes de calor por utilities calientes (Qs), uno en cada intervalo, al igual que de frías (Qw). De esta forma damos la oportunidad de que entren las utilities en cualquier nivel de temperatura. Creamos también i residuos (R), aunque sabemos que no habrá residuo del intervalo 0 ni tampoco del intervalo 9, lo cual implementaremos como dos restricciones diferentes.

In [148]:
Qs = model.Qs = Var(model.k,within = NonNegativeReals) 
Qw = model.Qw = Var(model.k,within = NonNegativeReals) 
R = model.R = Var(model.s,model.k,within = NonNegativeReals)
Q = model.Q = Var(model.s,model.w,model.k,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [149]:
model.util = Objective(expr = model.Qs[1] + model.Qw[intmax])

Constraints

In [150]:
nk = list(model.k)[1:] #Creamos una lista que contenga los índices sobre los que haremos el balance. En nuestro caso el balance lo aplicamos en todos los intervalos, excepto el intervalo 0. Es decir sobre 1-5, lo cual expresamos por [1:], es decir, desde 1 hasta el final de la lista.
model.C1 = ConstraintList()
for k in nk:
    for i in model.i:
        model.C1.add(
        R[i,k-1]+Hot_df.loc[i,k] == R[i,k]+sum(Q[i,j,k] for j in model.j)+sum(Q[i,w,k] for w in model.W)
        )

model.C2 = ConstraintList()
for k in nk:
    for s in model.S:
        model.C2.add(
        R[s,k-1]+Qs[k] == R[s,k]+sum(Q[s,j,k] for j in model.j)
        )        

model.C3 = ConstraintList()
for k in nk:
    for j in model.j:
        model.C3.add(
        sum(Q[i,j,k] for i in model.i)+ sum(Q[i,j,k] for i in model.S) == Cold_df.loc[j,k]
        )

model.C4 = ConstraintList()
for k in nk:
        model.C4.add(
        sum(Q[i,'W',k] for i in model.i)  == Qw[k]
        )


model.C5 = ConstraintList()
for s in model.s:
    model.C5.add(
        R[s,0]==0
        )

model.C6 = ConstraintList()
for s in model.s:
    model.C6.add(
        R[s,intmax]==0
        )

nm = list(model.k)[0:1]+list(model.k)[2:]
model.C7 = ConstraintList()
for m in nm:
    model.C7.add(
        Qs[m]==0
        )

nn = list(model.k)[0:intmax]
model.C8 = ConstraintList()
for n in nn:
    model.C8.add(
        Qw[n]==0
        )

model.C9 = ConstraintList()
for k in model.k:
    model.C9.add(
        Q['S','W',k] ==0
    )

model.C10 = ConstraintList()
for k in nk:
        model.C10.add(
        sum(Q[i,j,0] for i in model.i for j in model.j)  == 0
        )

Resolvemos el modelo

In [151]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

21 Set Declarations
    C10_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   40 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
    C4_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   1

In [152]:
Qss = value(model.Qs[1])
Qww = value(model.Qw[intmax])
print('Cold utility = {0:2.2f}, Hot utility = {1:2.2f}'.format(Qww, Qss))

Cold utility = 1195.00, Hot utility = 4631.55


Ahora resolvemos el problema de mínimo número de intercambios

In [153]:
U = {'C1': [4000, 1030,1290,880,1860], 'C2': [4496.55, 1030,1290,880,1860], 'W':[0,1030,1195,880,1195]}
U_df = pd.DataFrame(U, index=['S', 'H1', 'H2','H3','H4'])
U_df

,C1,C2,W
S,4000,4496.55,0
H1,1030,1030.00,1030
H2,1290,1290.00,1195
H3,880,880.00,880
H4,1860,1860.00,1195


In [154]:
model.y=Var(model.s,model.w, within=Binary)

model.C11 = ConstraintList()
for s in model.s:
    for w in model.w:
        model.C11.add(
        sum(Q[s,w,k] for k in model.k)-U_df.loc[s,w]*model.y[s,w] <=0
        )    

In [155]:
model.util.deactivate()
model.sumy = Objective(expr=sum(model.y[s,w] for s in model.s for w in model.w))

In [156]:
results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7.0
  Upper bound: 7.0
  Number of objectives: 1
  Number of constraints: 147
  Number of variables: 258
  Number of nonzeros: 701
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.08628273010253906
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [157]:
model.pprint()

23 Set Declarations
    C10_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C11_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   15 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   40 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   20 : {1, 2, 3, 4, 5,

In [158]:
optimal_values = {(i, j,k): value(model.Q[i,j,k]) for i in model.hot_st for j in model.cold_w for k in model.k}
df=pd.DataFrame.from_dict(optimal_values, orient="index", columns=['Q'])
df.to_excel('Datos/Resultados_Ejercicio4.xlsx', sheet_name="Results_Q")